In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)
import numpy as np
import matplotlib.pyplot as plt
from feature_manager import FeatureManager
import classifier.multi_dnn_classifier as dnn
from random import randint
from keras import callbacks
from keras import losses
from visualizer import *
from keras.utils import np_utils
from sklearn.metrics import classification_report
import importlib


2023-01-18 00:26:37.118797: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-01-18 00:26:37.118903: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-01-18 00:26:37.118916: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.


In [2]:
timeframe_in_ms = {"15m":15*60*1000,"1h":60*60*1000,"1d":24*60*60*1000}

symbol = "BTCUSDT"
trade_tf = "1h"
granular_tf = "1m"

tp = 0.05
sl = 0.025
md = 14


data = pd.read_csv("../data/{}-{}.csv".format(
    symbol,trade_tf), 
    parse_dates=["Open Time"], 
    index_col = "Open Time"
)

granular_data = pd.read_csv("../nocommit/{}-{}.csv".format(
    symbol,granular_tf), 
    parse_dates=["Open Time"], 
    index_col = "Open Time"
)

fm: FeatureManager = FeatureManager()
fm.build_feature(data=data,lags=5)

fm.prepare_trade_forward_data(
    data=fm.df,
    take_profit_rate=tp,
    stop_loss_rate=sl,
    max_duration=md,
    granular_data=granular_data,
    timeframe_in_ms=timeframe_in_ms[trade_tf]
)
# loop_classifier(data=fm.df,cols=fm.cols,target_col="trade_signal",laps=3)

Processing 1/14
Processing 2/14
Processing 3/14
Processing 4/14
Processing 5/14
Processing 6/14
Processing 7/14
Processing 8/14
Processing 9/14
Processing 10/14
Processing 11/14
Processing 12/14
Processing 13/14
Processing 14/14


In [3]:
fm.df["trade_signal"].value_counts()

0    38554
2     4379
1     3895
Name: trade_signal, dtype: int64

In [7]:
classifier = dnn.MultiClassDNNClassifer()
classifier.prepare_data(
    data = fm.df,
    cols = fm.cols,
    target_col="trade_signal",
    y_to_categorical=True, 
    rebalance="under",
    random_state=randint(0,100))
classifier.configure(hu=100,input_dim=len(fm.cols),class_num=3)
classifier.run()

Epoch 1/200

Epoch 1: val_loss improved from inf to 1.05281, saving model to ../logs/model/model_dnn_checkpoint.h5
818/818 - 3s - loss: 1.0681 - accuracy: 0.4373 - val_loss: 1.0528 - val_accuracy: 0.4576 - 3s/epoch - 4ms/step
Epoch 2/200

Epoch 2: val_loss improved from 1.05281 to 1.03655, saving model to ../logs/model/model_dnn_checkpoint.h5
818/818 - 2s - loss: 1.0077 - accuracy: 0.4905 - val_loss: 1.0365 - val_accuracy: 0.4815 - 2s/epoch - 3ms/step
Epoch 3/200

Epoch 3: val_loss improved from 1.03655 to 1.02445, saving model to ../logs/model/model_dnn_checkpoint.h5
818/818 - 2s - loss: 0.9885 - accuracy: 0.5081 - val_loss: 1.0245 - val_accuracy: 0.4863 - 2s/epoch - 3ms/step
Epoch 4/200

Epoch 4: val_loss did not improve from 1.02445
818/818 - 2s - loss: 0.9750 - accuracy: 0.5209 - val_loss: 1.0246 - val_accuracy: 0.4964 - 2s/epoch - 3ms/step
Epoch 5/200

Epoch 5: val_loss improved from 1.02445 to 1.01600, saving model to ../logs/model/model_dnn_checkpoint.h5
818/818 - 2s - loss: 0.9

In [6]:
importlib.reload(dnn)

<module 'classifier.multi_dnn_classifier' from '/home/andy/CryptoTradingPlatform/TraderRobot/project/classifier/multi_dnn_classifier.py'>